In [1]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%run scripts/helper.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')
crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [4]:
# structure of the training and test dataset
crowd_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10158 entries, 1 to 32668
Data columns (total 5 columns):
query                  10158 non-null object
product_title          10158 non-null object
product_description    10158 non-null object
median_relevance       10158 non-null int64
relevance_variance     10158 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 357.1+ KB


In [5]:
# structure of the test dataset
crowd_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22513 entries, 3 to 32671
Data columns (total 3 columns):
query                  22513 non-null object
product_title          22513 non-null object
product_description    22513 non-null object
dtypes: object(3)
memory usage: 439.7+ KB


### See what these two median relevance and relevance variance columns are all about

In [6]:
# training data
crowd_train.median_relevance.describe()

count    10158.000000
mean         3.309805
std          0.980666
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          4.000000
Name: median_relevance, dtype: float64

In [7]:
crowd_train.relevance_variance.describe()

count    10158.000000
mean         0.377863
std          0.389707
min          0.000000
25%          0.000000
50%          0.471000
75%          0.471000
max          1.470000
Name: relevance_variance, dtype: float64

### Mean of median relevance is around 3 which means that most of the search results were in fact relevant

### Lets analyze the corpus

In [10]:
# lets concatenate all the query values
queries = ' '.join([crowd_train.iloc[i].query for i in range(crowd_train.shape[0])])

In [13]:
# lowercase all queries and remove html tags
queries = queries.lower()
queries = queries.replace('<', '')
queries = queries.replace('>', '')

In [35]:
from nltk.corpus import stopwords

def content_fraction(text):
    content = [w for w in text if w not in stopwords.words('english')]
    return len(content)* 1. / len(text)

In [36]:
content_fraction(queries.split(' '))

0.9740164832934107